### Data Augmentation and Oversampling

The idea is to leverage the metadata and following the alogrithm proposed by Hashemi et al. (2023) to get pairs of non-consecutive paragraphs (with and
without style changes). Then, classes will be oversampled to obtain a balanced data set.

Description of algorithm: " incorporate additional non-consecutive pairs of paragraphs
into our sample set and assign them labels based on the inferred relationships. For example, if
there are three consecutive paragraphs without a style change, we can infer that the first and
third paragraphs are written by the same author. Similarly, if there are style changes between
the first and second paragraphs and between the second and third paragraphs, we can deduce
that the authors of the first and third paragraphs are different, given that the number of authors
in the document exceeds the number of style changes by one." (Hashemi et al. 2023: 4).

In [27]:
import os, json
import pandas as pd

In [24]:
BASE_DIR = '../data_pipeline/'

# get data sets 
df_train = pd.read_csv(os.path.join(BASE_DIR, "df_train.csv"))
df_val = pd.read_csv(os.path.join(BASE_DIR, "df_validation.csv"))

# add a column for number of authors in file as needed by algorithm --> changed it in data pipeline

# check distribution of labels
changes_train = len(df_train[df_train['label_author'] == 1])
no_changes_train = len(df_train[df_train['label_author'] == 0])

changes_val = len(df_val[df_val['label_author'] == 1])
no_changes_val = len(df_val[df_val['label_author'] == 0])

print(f"Number of rows where label_author == 0 (training data): {no_changes_train}")
print(f"Number of rows where label_author == 1 (training data): {changes_train}")

print(f"Number of rows where label_author == 0 (validation data): {no_changes_val}")
print(f"Number of rows where label_author == 1 (validation data): {changes_val}")

Number of rows where label_author == 0 (training data): 20485
Number of rows where label_author == 1 (training data): 31508
Number of rows where label_author == 0 (validation data): 4489
Number of rows where label_author == 1 (validation data): 6709


In [26]:
def data_augmentation(data):
    augmented_df = data
    for i in range(data[fileindex].loc[-1]): # iterate over fileindexes 
        
    pass
df_train[0:10]

,Unnamed: 0,paragraph1,paragraph2,label_author,label_dataset,fileindex
0,0,"In general, be courteous to others. Debate/dis...",My goodness. The poor woman can’t even get her...,1,0,1
1,1,My goodness. The poor woman can’t even get her...,"Exactly. If Sicknik had ""recovered"" that day a...",1,0,1
2,2,"Exactly. If Sicknik had ""recovered"" that day a...",r/politics is currently accepting new moderato...,1,0,1
3,3,r/politics is currently accepting new moderato...,"Essentially, if someone commits a tort, like a...",1,0,1
4,4,"Essentially, if someone commits a tort, like a...","I am a bot, and this action was performed auto...",1,0,1
5,5,I read Bottle of Lies by Katherine Eban last y...,"Actually, it is relevant because A) Reddit is ...",1,0,2
6,6,"Actually, it is relevant because A) Reddit is ...","Good, those sick fucks. I grew up in HK, and e...",1,0,2
7,7,"Good, those sick fucks. I grew up in HK, and e...",It’s definitely a loophole that should be clos...,1,0,2
8,8,It’s definitely a loophole that should be clos...,Yep. Actually it wasn't last year I was lookin...,1,0,2
9,9,"Like others, it made more sense for my husband...",Not particularly. In rural Missouri the local ...,1,0,3


In [ ]:
def swap_order(data):
    '''double the training data by swapping paragraph1 and paragraph2'''
    pass